In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import pickle as pk
import json

In [ ]:
path_git = 'd:/Git_Proyects/Tesis/Base_Twitter/'

nombre = 'kici'

kici_oct = pd.read_csv(path_git+'Tweets_'+nombre+'_oct.csv')
clusters = pk.load(open(path_git+nombre+'_clusters_75_oct.pk', 'rb'))

frases = list(kici_oct['Tweets_sin_url'])
cluster_map = {frases[j]: str(i+1) for i, clus in enumerate(clusters) for j in clus}
kici_oct['cluster'] = kici_oct['Tweets_sin_url'].map(cluster_map)
kici_oct['RT'] = False

df_cluster = kici_oct[kici_oct['cluster'] == True]


In [ ]:
import json

filename = 'd:/Facultad/Tesis/Twitter/201910-kicillof.txt'
tweets_cluster = set(list(df_cluster['Tweets']))

# Crea un diccionario para almacenar los tweets originales y sus retweets asociados
retweets_dict = {tweet: [] for tweet in tweets_cluster}

retweets_data = []
with open(filename, 'r', encoding='utf8') as fp:
    for i, line in enumerate(fp):
        json_data = json.loads(line)
        tweet = json_data['text']
        if tweet.startswith('RT'):
            try:
                original_tweet = json_data['retweeted_status']['full_text']
                if original_tweet in tweets_cluster:
                    retweets_dict[original_tweet].append(tweet)
                    filt = df_cluster['Tweets'] == original_tweet
                    verifide = df_cluster[filt]['verified'].tolist()[0]
                    retweets_data.append({
                        "Tweets": original_tweet,
                        "Fecha": json_data['created_at'],
                        "urls": 0,
                        "verified": verifide,
                        "Tweets_sin_url": original_tweet,
                        "cluster": '3',
                        "RT": True
                    })               
            except KeyError:
                pass
df_cluster.drop(columns=['Unnamed: 0'], inplace=True)


In [ ]:
df_RT = pd.DataFrame(retweets_data)
clus_3_completo = pd.concat([df_cluster, df_RT], ignore_index=True

In [ ]:
clus_3_completo['Fecha'] = pd.to_datetime(clus_3_completo['Fecha'], format='%a %b %d %H:%M:%S +0000 %Y')
horas = '6H'
#ventana_temporal = pd.offsets.Hour(horas)
clus_3_medios =  clus_3_completo[clus_3_completo['medios']== True]
clus_3_user =  clus_3_completo[clus_3_completo['usuarios']== False]

dos_tipos =[clus_3_medios, clus_3_user]
names = ['verificados', 'usuarios']
plt.figure()

for i, tipos in enumerate(dos_tipos):
    df_grouped = tipos.groupby(pd.Grouper(key='Fecha', freq=horas)).size().reset_index(name='count')
    plt.plot(df_grouped['Fecha'], df_grouped['count'], marker='o', linestyle='-', label = names[i])

# df_grouped = clus_3_completo.groupby(pd.Grouper(key='Fecha', freq=horas)).size().reset_index(name='count')
# plt.plot(df_grouped['Fecha'], df_grouped['count'], marker='o', linestyle='-', label = 'completo')
plt.title("Tweets y RT ventana 6hs")
plt.xlabel('Fecha y hora')
plt.ylabel('Cantidad de elementos')
#plt.xlim([datetime(2019, 10, 26), datetime(2019, 10, 30)])
plt.yscale('log')
plt.legend()
plt.grid(alpha = 0.7)
plt.xticks(rotation=45)
plt.show()